In [4]:
x = 3
print(id(x))

94630459120224


In [5]:
y = x
print (id(y))

94630459120224


In [16]:
string  = "Kem cho?"
len(string)
type(string)
type(string[-1])
string[0:]
#strings are immutable
#string[-1] = "p"


'Kem cho?'

In [19]:
name = input("What's your name? ")

What's your name? "Bgaksk"


In [20]:
print("Nice to meet you " + name + "!")

Nice to meet you "Bgaksk"!


In [21]:
languages = ["C", "C++", "Perl", "Python"] 

In [22]:
for x in languages:
    print(x)

C
C++
Perl
Python


In [23]:
import numpy as np

In [25]:
x = np.zeros((2, 3))

In [32]:
x[0: , 0]

array([0., 0.])